# Shortwave Radiative Transfer Example with PyRTE-RRTMGP

This notebook demonstrates how to use the PyRTE-RRTMGP package to solve a shortwave radiative transfer problem. PyRTE-RRTMGP is a Python implementation of the Radiative Transfer for Energetics (RTE).

## Overview

PyRTE-RRTMGP provides a flexible and efficient framework for computing radiative fluxes in planetary atmospheres. This example specifically focuses on:

1. Loading gas optics data for shortwave radiation
2. Processing atmospheric data from the RFMIP (Radiative Forcing Model Intercomparison Project)
3. Computing gas optics properties
4. Solving the radiative transfer equation to obtain upward and downward fluxes
5. Validating results against reference solutions generated with the original RTE fortran code

The package leverages xarray and dask for efficient data handling and parallel computation, making it suitable for large-scale atmospheric modeling applications.

## Key Components

- **Gas Optics**: Handles spectral properties of atmospheric gases
- **RTE Solver**: Computes radiative fluxes based on atmospheric properties
- **Data Handling**: Uses xarray for labeled, multi-dimensional data structures

This example demonstrates the workflow for shortwave radiative transfer calculations, which are essential for understanding Earth's energy budget and climate modeling.

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.


## Setup and Configuration

First, we import the necessary libraries and modules. PyRTE-RRTMGP relies on:
- **numpy** and **xarray** for data handling
- Various modules from the `pyrte_rrtmgp` package used to load the data

The key components we'll use are:
- `rrtmgp_gas_optics`: Handles spectral properties of gases
- `rte_solver`: Solves the radiative transfer equation

In [1]:
import numpy as np

from pyrte_rrtmgp.data_types import RFMIPExampleFiles, GasOpticsFiles
from pyrte_rrtmgp.utils import load_rrtmgp_file
from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rte_solver import rte_solve

## Loading Gas Optics Data

We get the default data files from the package that are available in the [rrtmgp-data](https://github.com/earth-system-radiation/rrtmgp-data) repository.

We're using the shortwave gas optics file with 224 g-points (`SW_G224`).

The atmosphere is the RFMIP (Radiative Forcing Model Intercomparison Project) dataset. This dataset contains atmospheric profiles with various combinations of temperature, pressure, and gas concentrations.

In [2]:
gas_optics_sw = rrtmgp_gas_optics.load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)
atmosphere = load_rrtmgp_file(RFMIPExampleFiles.RFMIP)

# Computing Gas Optics

Next, we define the gas mapping dictionary that specifies the gas names in the atmosphere dataset, the gas optics file names are the keys in the dictionary and are the default names used internally in the package.

With that we compute the gas optics for the atmosphere `two-stream` problem type. The computed gas optics are stored in the `atmosphere` Dataset.

In [3]:
gas_mapping = {
    "h2o": "water_vapor",
    "co2": "carbon_dioxide_GM",
    "o3": "ozone",
    "n2o": "nitrous_oxide_GM",
    "co": "carbon_monoxide_GM",
    "ch4": "methane_GM",
    "o2": "oxygen_GM",
    "n2": "nitrogen_GM",
    "ccl4": "carbon_tetrachloride_GM",
    "cfc11": "cfc11_GM",
    "cfc12": "cfc12_GM",
    "cfc22": "hcfc22_GM",
    "hfc143a": "hfc143a_GM",
    "hfc125": "hfc125_GM",
    "hfc23": "hfc23_GM",
    "hfc32": "hfc32_GM",
    "hfc134a": "hfc134a_GM",
    "cf4": "cf4_GM",
    "no2": "no2",
}

gas_optics_sw.compute_gas_optics(atmosphere, problem_type="two-stream", gas_name_map=gas_mapping)
atmosphere["tau"]

<xarray.DataArray 'tau' (site: 100, expt: 18, layer: 60, gpt: 224)> Size: 194MB
array([[[[2.92835921e-08, 2.50177483e-07, 7.05515885e-07, ...,
          2.15569396e-02, 2.15878679e-02, 2.15966541e-02],
         [1.87398004e-08, 1.14210575e-07, 2.53919195e-07, ...,
          5.16979458e-02, 5.17742947e-02, 5.17814588e-02],
         [2.16777890e-08, 1.09423192e-07, 2.50578566e-07, ...,
          1.06670533e-01, 1.06794753e-01, 1.06818004e-01],
         ...,
         [2.04913281e-03, 3.36736680e-03, 7.77122638e-03, ...,
          2.45832772e-01, 2.45833225e-01, 2.45528570e-01],
         [1.43651530e-03, 2.34868183e-03, 5.45754478e-03, ...,
          1.71239972e-01, 1.71240288e-01, 1.71016220e-01],
         [9.72027187e-04, 1.58432848e-03, 3.70404006e-03, ...,
          1.15151731e-01, 1.15151943e-01, 1.14998246e-01]],

        [[2.30143799e-08, 2.04747131e-07, 5.74482335e-07, ...,
          2.33621436e-02, 2.33955980e-02, 2.34050000e-02],
         [1.55703235e-08, 9.33767538e-08, 2.11782309e-07, ...,
          5.45246111e-02, 5.46050645e-02, 5.46126278e-02],
         [1.80659745e-08, 8.96811879e-08, 2.09179628e-07, ...,
          1.10687052e-01, 1.10815847e-01, 1.10839860e-01],
...
         [2.63282748e-03, 4.99243721e-03, 1.28675817e-02, ...,
          3.16364599e-01, 3.16365125e-01, 3.16041463e-01],
         [1.82188994e-03, 3.46438589e-03, 8.91220290e-03, ...,
          2.16657283e-01, 2.16657644e-01, 2.16447336e-01],
         [1.24961803e-03, 2.39350066e-03, 6.09599703e-03, ...,
          1.45183820e-01, 1.45184062e-01, 1.45048822e-01]],

        [[1.79850402e-08, 1.54603133e-07, 4.26339205e-07, ...,
          2.30367529e-02, 2.30701749e-02, 2.30791289e-02],
         [1.15512206e-08, 6.13165416e-08, 1.34729221e-07, ...,
          5.38594245e-02, 5.39352427e-02, 5.39418443e-02],
         [1.40042376e-08, 6.44976610e-08, 1.59292121e-07, ...,
          1.11110814e-01, 1.11228051e-01, 1.11255904e-01],
         ...,
         [3.96933740e-03, 4.71314220e-03, 8.71111371e-03, ...,
          3.17198845e-01, 3.17199372e-01, 3.16838715e-01],
         [2.74077701e-03, 3.25366490e-03, 6.05088840e-03, ...,
          2.17231362e-01, 2.17231723e-01, 2.16996519e-01],
         [1.86386259e-03, 2.22994635e-03, 4.16528031e-03, ...,
          1.45571872e-01, 1.45572114e-01, 1.45420388e-01]]]])
Dimensions without coordinates: site, expt, layer, gpt

## Solving the Radiative Transfer Equation

With the gas optics properties computed, we can now solve the radiative transfer equation using the `rte_solve`. This will calculate the upward and downward shortwave radiative fluxes for each atmospheric profile.

In [4]:
fluxes = rte_solve(atmosphere, add_to_input=False)
fluxes

<xarray.Dataset> Size: 3MB
Dimensions:       (site: 100, expt: 18, level: 61)
Coordinates:
  * site          (site) int64 800B 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * expt          (expt) int64 144B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
Dimensions without coordinates: level
Data variables:
    sw_flux_up    (site, expt, level) float64 878kB 129.1 129.1 ... 30.93 30.85
    sw_flux_down  (site, expt, level) float64 878kB 757.4 757.2 ... 514.7 514.1
    sw_flux_dir   (site, expt, level) float64 878kB 757.4 757.1 ... 467.1 466.5

## Validating Results Against Reference Solutions

Finally, we validate our computed fluxes against reference solutions. The reference data comes from the original RTE-RRTMGP implementation and makes sure that the implementation is correct.

We compare both upward (`rsu`) and downward (`rsd`) shortwave fluxes to ensure our implementation produces accurate results within the specified error tolerance (`ERROR_TOLERANCE = 1e-7`).


In [5]:
rsu = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RSU)
rsd = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RSD)

assert np.isclose(fluxes["sw_flux_up"].transpose("expt", "site", "level"), rsu["rsu"], atol=1e-7).all(), "Shortwave flux up mismatch"
assert np.isclose(fluxes["sw_flux_down"].transpose("expt", "site", "level"), rsd["rsd"], atol=1e-7).all(), "Shortwave flux down mismatch"

print("Shortwave radiation transfer calculations validated successfully!")

Shortwave radiation transfer calculations validated successfully!
